In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm
import umap.umap_ as umap

2026-01-18 16:38:21.389268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768754301.660151      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768754301.738711      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## Предобработка

In [2]:
data = pd.read_csv('/kaggle/input/food-data/food_data.csv')
print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2679 entries, 0 to 2678
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_name        2594 non-null   object 
 1   category            2673 non-null   object 
 2   energy_kcal_100g    2679 non-null   float64
 3   fat_100g            2679 non-null   float64
 4   sugars_100g         2678 non-null   float64
 5   carbohydrates_100g  2679 non-null   float64
 6   serving_size_g      2649 non-null   float64
 7   packaging_quantity  41 non-null     float64
 8   source_url          2679 non-null   object 
dtypes: float64(6), object(3)
memory usage: 188.5+ KB
None


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,product_name,category,energy_kcal_100g,fat_100g,sugars_100g,carbohydrates_100g,serving_size_g,packaging_quantity,source_url
0,Jben,"en:Dairies, en:Fermented foods, en:Fermented m...",235.0,21.0,0.0,3.50,160.0,NaN,https://world.openfoodfacts.org/product/611124...
1,Marmite Yeast Extract,"Dairies, Fermented foods, Fermented milk produ...",260.0,0.5,1.2,30.00,8.0,NaN,https://world.openfoodfacts.org/product/50184453
2,NaN,"Produits laitiers,Produits fermentés,Produits ...",101.0,6.0,0.0,4.20,90.0,NaN,https://world.openfoodfacts.org/product/611124...
3,Prince Goût Chocolat,"Snacks,Snacks sucrés,Biscuits et gâteaux,Biscu...",467.0,17.0,32.0,69.00,20.0,NaN,https://world.openfoodfacts.org/product/762221...
4,Nutella,"Petit-déjeuners,Produits à tartiner,Produits à...",539.0,30.9,56.3,57.50,15.0,NaN,https://world.openfoodfacts.org/product/301762...
5,Tonik,"Plant-based foods and beverages,Plant-based fo...",504.0,22.2,40.0,71.70,23.0,NaN,https://world.openfoodfacts.org/product/611103...
6,Sésame,"Snacks,Snacks sucrés,Biscuits et gâteaux,Biscu...",467.0,18.0,17.0,64.00,11.5,NaN,https://world.openfoodfacts.org/product/317568...
7,Confiture de fraise,"Aliments et boissons à base de végétaux,Alimen...",250.0,0.3,63.0,61.33,430.0,NaN,https://world.openfoodfacts.org/product/611116...
8,Yaourt nature,"Produits laitiers, Produits fermentés, Dessert...",78.5,3.3,0.0,8.00,100.0,NaN,https://world.openfoodfacts.org/product/611124...
9,Dark Chocolate 85% Cocoa,"Snacks,Sweet snacks,Cocoa and its products,Cho...",579.0,46.8,12.9,21.40,12.5,NaN,https://world.openfoodfacts.org/product/20995553


In [3]:
data['product_name'] = data['product_name'].fillna('unknown')
data['category'] = data['category'].fillna(data['category'].mode()[0])
data['packaging_quantity'] = data['packaging_quantity'].fillna(1)
data['serving_size_g'] = data['serving_size_g'].fillna(100)

In [4]:
numeric_cols = [
    'energy_kcal_100g',
    'fat_100g',
    'sugars_100g',
    'carbohydrates_100g',
    'serving_size_g',
    'packaging_quantity'
]
data = data.dropna(subset=numeric_cols).reset_index(drop=True)
X = data[numeric_cols].copy()

# небольшой тримминг выбросов (опционально, но полезно)
for c in numeric_cols:
    low, high = X[c].quantile([0.01, 0.99])
    X[c] = X[c].clip(low, high)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(X)


In [5]:
features = numeric_cols
X = data[features].values
scaler = StandardScaler()
data_scaled = scaler.fit_transform(X)

# 1. Классические методы кластеризации

In [6]:
!pip install umap-learn==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82810 sha256=d792ddb7280cb7ba55f3f0d9970266c64ed1a721519f768f07b1a2b31ed05cdb
  Stored in directory: /root/.cache/pip/wheels/02/15/cd/b456fcce97d704f38b583aecf7fa174860b6ff0b499f51f95a
Successfully built umap-learn
  Attempting uninstall: umap-learn
    Found existing installation: umap-learn 0.5.9.post2
    Uninstalling umap-learn-0.5.9.post2:
      Successfully uninstalled umap-learn-0.5.9.post2


In [7]:
from umap import UMAP

umap_model = UMAP(
    n_components=2,
    random_state=42,
    metric='euclidean',
    n_neighbors=15,
    min_dist=0.1
)
X_umap = umap_model.fit_transform(data_scaled)
X_umap.shape

TypeError: check_array() got an unexpected keyword argument 'ensure_all_finite'

## 1.1 K-Means

In [ ]:
models = []
score1 = [] # функция стоимости
score2 = [] #значения коэффициента силуэта
for i in range(2, 20):
    model = KMeans(n_clusters=i, random_state=123, init='k-means++').fit(data_scaled)
    models.append(model)
    score1.append(model.inertia_)
    score2.append(silhouette_score(data_scaled, model.labels_))

In [ ]:
# график локтя
plt.grid()
plt.plot(np.arange(2, 20), score1, marker='o')
plt.title('Визуализация правила локтя')
plt.show()

In [ ]:
# коэффициент силуэта
plt.grid()
plt.plot(np.arange(2, 20), score2, marker='o', color='red')
plt.title('Визуализация коэффициента силуэта')
plt.show()

In [ ]:
best_k = 3

kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=20)
kmeans_labels = kmeans.fit_predict(data_scaled)

reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(data_scaled)

plt.figure(figsize=(6,5))
plt.scatter(X_umap[:,0], X_umap[:,1], c=kmeans_labels, cmap='tab10', s=10)
plt.title(f'K-Means, k={best_k}')
plt.xlabel('UMAP-1')
plt.ylabel('UMAP-2')
plt.colorbar(label='cluster')
plt.show()

print("Silhouette (K-Means):", silhouette_score(data_scaled, kmeans_labels))
data['cluster_kmeans'] = kmeans_labels


In [ ]:
model1 = KMeans(n_clusters=3, random_state=123, init='k-means++')
model1.fit(data_scaled)
model1.cluster_centers_

In [ ]:
labels_kmeans=model1.labels_

In [ ]:
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=labels_kmeans, cmap='viridis')
plt.title('K-Means')
plt.show()

## 1.2 Иерархическая кластеризация

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

n_samples = 500  # можно увеличить/уменьшить
if data_scaled.shape[0] > n_samples:
    rng = np.random.RandomState(42)
    idx = rng.choice(data_scaled.shape[0], n_samples, replace=False)
    X_for_dendro = data_scaled[idx]
else:
    X_for_dendro = data_scaled

linkage_matrix = linkage(X_for_dendro, method='ward')

plt.figure(figsize=(14, 7))
dendrogram(
    linkage_matrix,
    truncate_mode='level',  # показываем только последние уровни
    p=5,                    # число уровней
    leaf_font_size=10,
    show_leaf_counts=True,
    color_threshold=None    # пока без порога, посмотрим масштаб
)
plt.title('Дендрограмма')
plt.xlabel('Объекты / кластеры')
plt.ylabel('Расстояние (linkage distance)')
plt.tight_layout()
plt.show()

In [ ]:
agglom = AgglomerativeClustering(n_clusters=3)
agglom.fit(data_scaled)
labels_agglom = agglom.labels_

In [ ]:
plt.figure(figsize=(7, 6))
scatter = plt.scatter(
    X_umap[:, 0], X_umap[:, 1],
    c=labels_agglom,
    cmap='tab10',
    s=15,
    alpha=0.8
)
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.colorbar(scatter, label="Номер кластера")
plt.tight_layout()
plt.show()

## 1.3 DBSCAN

In [ ]:
def compute_silhouette(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(X)

    # Проверяем, есть ли хотя бы 2 кластера (без учёта шума)
    unique_labels = set(labels)
    n_clusters = len(unique_labels - {-1})

    if n_clusters < 2:
        return -1  # silhouette_score не определён → плохой результат

    try:
        score = silhouette_score(X, labels)
        return score
    except:
        return -1

# Диапазоны параметров
eps_values = np.arange(0.01, 50, 0.1)
min_samples_values = np.arange(1, 30, 1)

best_score = -1
best_params = {}

for eps in tqdm(eps_values, "eps"):
    for min_samples in tqdm(min_samples_values, "min_smples"):
        score = compute_silhouette(X, eps, min_samples)
        if score > best_score:
            best_score = score
            best_params = {"eps": eps, "min_samples": min_samples}

print("Best silhouette score:", best_score)
print("Best params:", best_params)

In [ ]:
#dbscan = DBSCAN(**best_params)
eps=47.51
min_samples=13
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X)
labels_dbscan = dbscan.labels_

In [ ]:
plt.figure(figsize=(7, 6))
scatter = plt.scatter(
    X_umap[:, 0], X_umap[:, 1],
    c=labels_dbscan,
    cmap='viridis',
    s=15
)
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.colorbar(scatter, label="Номер кластера")
plt.tight_layout()
plt.show()

# 2. Метод безразмерных комплексов

In [ ]:
fat = data['fat_100g'].values
carbs = data['carbohydrates_100g'].values
sugars = data['sugars_100g'].values
energy = data['energy_kcal_100g'].values
serv = data['serving_size_g'].values
packs = data['packaging_quantity'].values

eps = 1e-6

macros = fat + carbs
complex_carbs = np.maximum(carbs - sugars, 0)
theor_energy  = 9 * fat + 4 * carbs

complex_1 = np.where(macros > 0, fat / macros, 0.0)            # доля жира в макро
complex_2 = np.where(carbs > 0, sugars / carbs, 0.0)           # доля сахара в углеводах
complex_3 = np.where(theor_energy > 0, energy / theor_energy, 0.0)
complex_4 = np.where(macros > 0, sugars / macros, 0.0)         # доля сахара в макро
complex_5 = np.where(carbs > 0, complex_carbs / carbs, 0.0)    # сложн. углеводы / все углеводы
complex_6 = np.where(sugars > 0, fat / (sugars + eps), 0.0)    # жир / сахар
complex_7 = np.where(carbs > 0, fat / (carbs + eps), 0.0)      # жир / углеводы
complex_8 = serv / 100.0                                       # размер порции / 100 г
serving_energy = energy * serv / 100.0  # энергия одной порции (ккал в 1 порции)
package_energy = serving_energy * packs  # энергия всей упаковки (ккал во всей упаковке)
complex_10 = serving_energy / 2000.0    # доля суточной нормы энергии в одной порции (при норме 2000 ккал)
complex_11 = package_energy / 2000.0   # доля суточной нормы энергии во всей упаковке

In [ ]:
import itertools

fat    = data['fat_100g'].values
carbs  = data['carbohydrates_100g'].values
sugars = data['sugars_100g'].values
energy = data['energy_kcal_100g'].values
serv   = data['serving_size_g'].values
packs  = data['packaging_quantity'].values

eps = 1e-6

macros        = fat + carbs
complex_carbs = np.maximum(carbs - sugars, 0)      # углеводы без сахара
theor_energy  = 9 * fat + 4 * carbs               # теоретическая ккал по макро
serving_energy  = energy * serv / 100.0           # ккал в 1 порции
package_energy  = serving_energy * packs          # ккал во всей упаковке

complex_1  = np.where(macros > 0, fat / macros, 0.0)            # доля жира в макро
complex_2  = np.where(carbs > 0, sugars / carbs, 0.0)           # доля сахара в углеводах
complex_3  = np.where(theor_energy > 0, energy / theor_energy, 0.0)  # факт/теор энергия
complex_4  = np.where(macros > 0, sugars / macros, 0.0)         # доля сахара в макро
complex_5  = np.where(carbs > 0, complex_carbs / carbs, 0.0)    # сложные углеводы / все углеводы
complex_6  = serv / 100.0                                       # масса порции / 100 г
complex_7 = serving_energy / 2000.0                            # доля суточной нормы ккал в порции
complex_8 = package_energy / 2000.0                            # доля суточной нормы ккал в упаковке

complex_df = pd.DataFrame({
    "c1_fat_share_macro"             : complex_1,
    "c2_sugar_share_carbs"           : complex_2,
    "c3_energy_ratio_macro"          : complex_3,
    "c4_sugar_share_macro"           : complex_4,
    "c5_complex_carbs_share_carbs"   : complex_5,
    "c8_serving_mass_rel_100g"       : complex_6,
    "c9_serving_energy_daily_share" : complex_7,
    "c10_package_energy_daily_share" : complex_8,
}).dropna()

display(complex_df.head())

In [ ]:
# --- 3. Попарная визуализация всех комплексов ---
%matplotlib inline
cols = complex_df.columns
pairs = list(itertools.combinations(cols, 2))
n_pairs = len(pairs)

ncols = 3
nrows = int(np.ceil(n_pairs / ncols))

fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(5 * ncols, 4 * nrows),
    squeeze=False
)
axes = axes.flatten()

for idx, (x_col, y_col) in enumerate(pairs):
    ax = axes[idx]
    ax.scatter(complex_df[x_col], complex_df[y_col], s=1.0, alpha=0.5)
    ax.set_xlabel(x_col, fontsize=8)
    ax.set_ylabel(y_col, fontsize=8)
    ax.grid(True, linestyle="--", alpha=0.3)

# Удаляем лишние пустые оси, если они есть
for j in range(len(pairs), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()